In [2]:
from pprint import pprint
import json
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed

# Sending emails using imaplib and email libraries

In [3]:
import imaplib
import email

imap_server = "imap.gmail.com"
email_address = "appdevemail.test@gmail.com"
password = "lwvjycgyebbbxvwb" # App password not the actual password. This is necessary especially if 2-factor authentication is enabled

# TODO Connect and login to the gmail server and account
imap = imaplib.IMAP4_SSL(imap_server)
imap.login(email_address, password)

# TODO Select the inbox category
imap.select("Inbox")

# TODO Search the Inbox
# msg_nums = imap.search(None, "ALL")[1]
_, msg_nums = imap.search(None, "ALL")
# print(msg_nums)
# print("")

#TODO Convert the list into a conventional list
msg_nums_list = msg_nums[0].split()
# print(msg_nums_list)
# print("")

# TODO Get the actual message
code_standard = "(RFC822)"
for msg_num in msg_nums_list:
    data = imap.fetch(msg_num, code_standard)[1]
    # print(data)
    # TODO Make data readable
    message = email.message_from_bytes(data[0][1])
    # print(message)
    print(f"Message Number: {msg_num} \n"
          f"From: {message.get('From')} \n"
          f"To: {message.get('To')} \n"
          f"BCC: {message.get('BCC')} \n"
          f"Date: {message.get('Date')} \n"
          f"Subject: {message.get('Subject')} \n"
          f"Content: ")
    for part in message.walk(): # Generator that yields all parts of the message
        if part.get_content_type() == "text/plain":
            print(part.as_string())

    print("\n\n")

imap.close()

Message Number: b'1' 
From: Google Community Team <googlecommunityteam-noreply@google.com> 
To: appdevemail.test@gmail.com 
BCC: None 
Date: Mon, 10 Oct 2022 03:07:19 -0700 
Subject: Vincent, finish setting up your new Google Account 
Content: 
Content-Type: text/plain; charset="UTF-8"; format=flowed; delsp=yes
Content-Transfer-Encoding: base64

DQoNCkhpIFZpbmNlbnQsDQoNCldlbGNvbWUgdG8gR29vZ2xlLiBZb3VyIG5ldyBhY2NvdW50IGNv
bWVzIHdpdGggYWNjZXNzIHRvIEdvb2dsZSBwcm9kdWN0cywgIA0KYXBwcyBhbmQgc2VydmljZXMu
DQoNCkhlcmUgYXJlIGEgZmV3IHRpcHMgdG8gZ2V0IHlvdSBzdGFydGVkLg0KDQoNCkdldCB0aGUg
bW9zdCBvdXQgb2YgeW91ciBHb29nbGUgQWNjb3VudA0KDQpXZSdsbCBzZW5kIHlvdSBwZXJzb25h
bGlzZWQgdGlwcywgbmV3cyBhbmQgcmVjb21tZW5kYXRpb25zIGZyb20gR29vZ2xlLg0KDQpZZXMs
IGtlZXAgbWUgdXBkYXRlZCAgDQo8aHR0cHM6Ly9ub3RpZmljYXRpb25zLmdvb2dsZS5jb20vZy9w
L0FLV29MUWdpSzB3QmlfVDlxNThybVhDYkl6THluVGVlS0NpVUQ5OV92OGMyd1ZGcTdnbXhJVjZ0
YURTZXJ2X2hxVW1GRnZwUWZXR2R5dmNnVy1zZWxHTkluU1Awb2QxNkRIZ3UwbHFKRjhDOUFibkk3
dzRnTlUwWEVtMTJVNGVCQ2djRWs0MWozcV9B

('OK', [b'Returned to authenticated state. (Success)'])

# Configure Gmail API

In [4]:
# Run quickstart.py

# Get messages in a category using Gmail API

In [7]:
from __future__ import print_function

import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

"""Shows basic usage of the Gmail API.
Lists the user's Gmail labels.
"""
creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

# CATEGORY_PERSONAL, CATEGORY_SOCIAL, CATEGORY_PROMOTIONS, CATEGORY_UPDATES, CATEGORY_FORUMS
label_name = "CATEGORY_SOCIAL"
page_nums = 1
next_page_token = ''
message_id_list = []

for page in range(page_nums):
    # print(page + 1)
    try:
        # Call the Gmail API
        service = build('gmail', 'v1', credentials=creds)
        results = service.users().messages().list(userId='me', labelIds = [label_name], pageToken = next_page_token).execute()

        if len(results) == 0:
            break

    except HttpError as error:
        # TODO(developer) - Handle errors from gmail API.
        print(f'An error occurred: {error}')
    else:
        messages_details_list = results.get('messages')

        for message_details in messages_details_list:
            message_id_list.append(message_details['id'])
        # print(message_id_list)
        next_page_token = results.get('nextPageToken')

# print(len(message_id_list))
# print(service.users().messages().get(userId='me', id="184740d612e1b5a4").execute())


def get_message_content(message_id):
    service = build('gmail', 'v1', credentials=creds)
    result = service.users().messages().get(userId='me', id=message_id).execute()
    return result

message_list = []
# TODO Get the message for each of the message ids
with ThreadPoolExecutor(max_workers=len(message_id_list)) as executor:
    futures = [executor.submit(get_message_content, message_id) for message_id in message_id_list]

    some_dict = {}
    for future in as_completed(futures):
        message_data = future.result()
        message_id = message_data["id"]
        headers_list = message_data['payload']['headers']
        for elem in headers_list:
            if elem['name'] == "From":
                sender = elem['value']
                # print(sender)
            elif elem['name'] == "Subject":
                subject = elem['value']
                # print(subject)
            elif elem['name'] == "List-Unsubscribe":
                unsubscribe_link = elem['value'].replace("<", "").replace(">", "")
                # print(unsubscribe_link)

        if sender in some_dict:
            message_data_list = some_dict[sender]
            message_data_list.append(
                {"message_id": message_id,
                 'unsubscribe_link': unsubscribe_link,
                 'subject': subject
                 }
            )
        else:
                some_dict[sender] = [
                    {"message_id": message_id,
                     'unsubscribe_link': unsubscribe_link,
                     'subject': subject
                     }
                ]
        pprint(some_dict)
        message_list.append(some_dict)

        print("\n\n")






KeyboardInterrupt: 

# Search and Display emails using Gmail API

In [ ]:
some_dict = {
    'Twitter': [
        {
            'message_id': "dh923hfb327hgfihj24ghj",
            'unsubscribe_link': "http//najknfdjanjfnjadnfjnad"
        },
        {
            'message_id': "dh923hfb327hgfihj24ghj",
            'unsubscribe_link': "http//najknfdjanjfnjadnfjnad"
        }
    ],
    'Instagram': [
        {
            'message_id': "dh923dsfsdfhfb327hgfihj24ghj",
            'unsubscribe_link': "http//najknfd345uhjanjfnjadnfjnad"
        },
        {
            'message_id': "dh923hfbwefgrg327hgfihj24ghj",
            'unsubscribe_link': "http//najknfddsfwt4janjfnjadnfjnad"
        }
    ]
}

data_list = ['Twitter', "Pinterest", "Facebook", "Twitter"]
for sender in data_list:
    message_id = "dh923hfbwefgrg327hgfibgbhj24ghj"
    unsubscribe_link = "http//najknfddsfwt4sdfdsfjanjfnjadnfjnad"
    if sender in some_dict:
        message_data_list = some_dict[sender]
        message_data_list.append(
            {"message_id": message_id,
             'unsubscribe_link': unsubscribe_link
             }
        )
    else:
        some_dict[sender] = [
            {"message_id": message_id,
             'unsubscribe_link': unsubscribe_link
             }
        ]

pprint(some_dict)

for sender in data_list:
    message_id = "dh923hfbwefgrg327hgfibgbhj24ghj"
    unsubscribe_link = "http//najknfddsfwt4sdfdsfjanjfnjadnfjnad"
    if sender in some_dict:
        message_data_list = some_dict[sender]
        message_data_list.append(
            {"message_id": message_id,
             'unsubscribe_link': unsubscribe_link
             }
        )
    else:
        some_dict[sender] = [
            {"message_id": message_id,
             'unsubscribe_link': unsubscribe_link
             }
        ]

pprint(some_dict)


# Gmail using Selenium

In [ ]:
link = "\u003chttps://twitter.com/i/u?t\u003d1\u0026listunsub\u003dt\u0026cn\u003dYWN0aXZpdHlfZGlnZXN0X3dpdGhfaGVhZGxpbmVfY2hhbmdl\u0026sig\u003d99244ecee71b92d1a062f83740985c9e177ba49c\u0026iid\u003d2d788f4b4e8045e7ba6536741e14722e\u0026uid\u003d1424277263172898817\u0026nid\u003d244+26\u0026usbid\u003d16\u003e"
# decoded_link = link.replace("\u003d", "=")
# print(decoded_link)
print(link.replace("<", "").replace(">", ""))
"https://twitter.com/i/u?t=1&cn=ZmxleGlibGVfcmVjcw%3D%3D&sig=6d237d9001e5e1bff2b28e1f6aac1ecac4d13a48&iid=114d4e00390149e0810adfee8dfa037d&uid=1424277263172898817&nid=244+26&usbid=6"

In [ ]:
some_dict = {
    "Vincent": [1, 2, 3, 4],
    "Diana": [2, 4, 6, 8]
}

if "Vincent" in some_dict:
    some_dict["Vincent"].append()

